In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
import os
from diffusers import StableDiffusionPipeline
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # or try "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, token="")
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", token="")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

sd_model_id = "segmind/SSD-1B"
pipe2 = StableDiffusionPipeline.from_pretrained(
    sd_model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
).to("cuda:1")

# # Optionally set auth token for private models
# pipe.enable_xformers_memory_efficient_attention()

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:799: UserWarning: Not enough free disk space to download the file. The expected file size is: 4999.80 MB. The target location /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/blobs only has 0.00 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:799: UserWarning: Not enough free disk space to download the file. The expected file size is: 4976.70 MB. The target location /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/blobs only has 0.00 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:799: UserWarning: Not enough free disk space to download the file. The expected file size is: 4915.92 MB. The target location /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/blobs only has 0.00 MB free disk space.
  warnings.warn(
/usr/local

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

RuntimeError: Data processing error: CAS service error : IO Error: No space left on device (os error 28)

In [ ]:


user_input = "Generate a gym workout image prompt for training legs the prompt needs to be very extensive describe the action, the human and the surrounding as detailed as it could"
prompt = f"{user_input} in a professional cinematic lighting"

response = pipe(prompt, max_new_tokens=100)[0]["generated_text"]
print(response)
   
image = pipe2(response, num_inference_steps=30, guidance_scale=7.5).images[0]

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
image.save(f"output_{timestamp}.png")
# a
